In [7]:
class siamese_network:
    
    def __init__(self,height,width):
        '''
        Parameters:
        network_architecture: the architecture that will be used in the network
        '''
        self.width= width
        self.height= height
        
        #Using densenet architecuture pretrained on imagenet
        self.subnetwork= tf.keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet'\
                                                   ,input_tensor=None,input_shape=(self.height,self.width,3), pooling=None)
    
  
        
        
    def create_siamese_network(self):
        
        #Inputs for two images
        image1= Input(shape=(self.height,self.width,3))
        image2= Input(shape=(self.height,self.width,3))        
        
        #the output should be in the shape (batch,feature)
        output1= Flatten()(self.subnetwork(image1))
        output2= Flatten()(self.subnetwork(image2))
        
        #Finding the distance between the two
        #Therefore taking the square distance
        output = cosine_distance()(output1,output2)

        
        model= Model(inputs=[image1,image2],outputs=output)
        return model
    
    
    
    def util_plotting(self,train_precision,train_recall,test_precision,test_recall,epoch):
        
        train_evaluation= self.util_dataframe(train_precision,train_recall)
        test_evaluation= self.util_dataframe(test_precision,test_recall)
        
        #Plotting the precision and recall 
        fig, ax = plt.subplots(1,2,figsize=(14,6))
        plt.title('Train evaluationn')
        ax[0].set_title('Precision')
        ax[0].plot(train_evaluation['threshold'],train_evaluation['precision'],'ro--')
        ax[0].plot(test_evaluation['threshold'],test_evaluation['precision'],'bo--')
        ax[0].legend(['train','test'])
        ax[0].set_xlabel('Threshold')
        ax[0].set_ylabel('precision')
        
        ax[1].set_title('Recall')
        ax[1].plot(train_evaluation['threshold'],train_evaluation['recall'],'ro--')
        ax[1].plot(test_evaluation['threshold'],test_evaluation['recall'],'bo--')
        ax[1].legend(['train','test'])
        ax[1].set_xlabel('Threshold')
        ax[1].set_ylabel('recall')
        plt.show()
        fig.savefig(evaluation_dir+'fig_'+str(epoch)+'.png')
        
        return

    
    #Create dataframe of the evaluation metric
    def util_dataframe(self,precision_value,recall_value):
        
        threshold= list(precision_value.keys())
        dicti={'threshold':threshold,'precision':list(precision_value.values()),'recall':list(recall_value.values())}
        dataframe= pd.DataFrame(dicti)
        return dataframe
        
        
    def get_result_and_reset(self):
        
        #Creating a dictionary of precision and recall
        Precision={'0':precision_0,'0.2':precision_0_2,'0.4':precision_0_4,'0.6':precision_0_6,'0.8':precision_0_8}
        Recall= {'0':recall_0,'0.2':recall_0_2,'0.4':recall_0_4,'0.6':recall_0_6,'0.8':recall_0_8}
#         auc_value= auc.result.numpy()

        #Getting the value of preicion and recall at different threshold
        precision_values= {i:Precision[i].result().numpy() for i in list(Precision.keys())}
        recall_values= {i:Recall[i].result().numpy() for i in list(Recall.keys())}
        
        #reseting the metrics

        precision_0.reset_state()
        precision_0_2.reset_state()
        precision_0_4.reset_state()
        precision_0_6.reset_state()
        precision_0_8.reset_state()

        recall_0.reset_state()
        recall_0_2.reset_state()
        recall_0_4.reset_state()
        recall_0_6.reset_state()
        recall_0_8.reset_state()
        return precision_values,recall_values
        
    
    #Getting the gradient using tf.GradientTape and updating the wieghts of the model
    @tf.function
    def train_step(self,image,label):

        with tf.GradientTape() as tape:
            prediction = model(image, training=True)
            loss_value =contractive_loss(label,prediction)
            
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        #updating the metrics with new labels
#         auc.update_state(label,prediction)
        
        precision_0.update_state(label,prediction)
        precision_0_2.update_state(label,prediction)
        precision_0_4.update_state(label,prediction)
        precision_0_6.update_state(label,prediction)
        precision_0_8.update_state(label,prediction)

        recall_0.update_state(label,prediction)
        recall_0_2.update_state(label,prediction)
        recall_0_4.update_state(label,prediction)
        recall_0_6.update_state(label,prediction)
        recall_0_8.update_state(label,prediction)

        return loss_value
    
    
    
    #Creating static graph in tensorflow
    @tf.function
    def test_step(self,image,label):
        
        prediction = model(image, training=False)
        loss_value =contractive_loss(label,prediction)
        
        precision_0.update_state(label,prediction)
        precision_0_2.update_state(label,prediction)
        precision_0_4.update_state(label,prediction)
        precision_0_6.update_state(label,prediction)
        precision_0_8.update_state(label,prediction)

        recall_0.update_state(label,prediction)
        recall_0_2.update_state(label,prediction)
        recall_0_4.update_state(label,prediction)
        recall_0_6.update_state(label,prediction)
        recall_0_8.update_state(label,prediction)
        return loss_value

        
    def train_siamese_network(self,epochs,load_previous_model= True):
        
        '''
        Augments:
        epochs= no of epoch to train the model
        train_dataset= tensorflow dataset of train
        test_dataset= tensorflow dataset of test
        dir_to_save_checkpoints= path to the directory where the checkpoints are going to be saved
        dir_to_save_evaluation= path to the directory where evaluation dataframe is going to be saved
        '''
        
        self.epochs= epochs
         
        #Creating an checkpoint object and manager to keep track of the checkpoint
        Checkpt= Checkpoint(root= model)
        manager= tf.train.CheckpointManager(Checkpt,checkpoints_dir,3, checkpoint_name='./tf_ckpts')
                     
        previous_epoch= 0
        
        if load_previous_model== True:
            
            #Loading the previous checkpoints so that to start the training from that epoch             
            Checkpt.restore(manager.latest_checkpoint)
                   
            if manager.latest_checkpoint:
                previous_epoch= int(manager.latest_checkpoint.split('\\')[-1].split('-')[-1])
                print('Loading the model parameters from previous epoch ie:',previous_epoch)
                print('\n')
        else:
            
            #Deleting the previous checkpoints directory
            shutil.rmtree('./Checkpoints')
            !mkdir './Checkpoints'

     
        
        #Training the model using custom training loop 
        for epoch in range(self.epochs):
                start= datetime.now()

             # Iterate over the batches of the dataset
            
                for step,[image,label] in enumerate(train_dataset):
                    
                    #Calulating loss at each batch
                    loss_value= self.train_step(image,tf.cast(label,dtype=tf.float32))
                    
                    if step%100==0:
                        print('Epoch: ',epoch+previous_epoch,'/',self.epochs+previous_epoch)
                        print(step,'/',train_steps,'====================== loss: ',loss_value.numpy())
     
                #Calculate the value of precision and recall
                train_precision,train_recall= self.get_result_and_reset()
                

            # Run a validation loop at the end of each epoch.
            
                test_loss= 0
                for image,label in test_dataset:
                    loss_value= self.test_step(image,label)
                   
                    
                #Calculate the value of precision and recall
                test_precision,test_recall= self.get_result_and_reset()
                print('The test loss at epoch :',epoch+previous_epoch,'is :',loss_value)
                
                #Saving the result of the matrics
                self.util_plotting(train_precision,train_recall,test_precision,test_recall,epoch)
                
                #Saving the checkpoints
                manager.save()
                print('Model is saved')


In [53]:
#Parameters
epochs= 20
height= 288
width= 432
contractive_loss_parameter= 5.0
train_csv= './train.csv'
test_csv= './test.csv'
checkpoints_dir= './Checkpoints/'
# evaluation_dir= '/content/drive/MyDrive/Speaker identification'
evaluation_dir= './evaluation/'

# evaluation_dir= r'D:\Work\Speaker classification'
batch=16
train= pd.read_csv(train_csv)
test= pd.read_csv(test_csv)

train_steps= train.shape[0]//batch
test_steps= test.shape[0]//batch


In [95]:
ax

(2,)

In [20]:
list(eval_train.loc[0][0:5].values)

[0.10807888, 0.86249286, 0.90903205, 0.93401015, 0.95519865]